<img src=https://audiovisuales.icesi.edu.co/assets/custom/images/ICESI_logo_prin_descriptor_RGB_POSITIVO_0924.jpg width=200>

*Milton Orlando Sarria Paja, PhD.*

----

# 🛠️ Metodologías para validar un clasificador

Ajustar los parámetros de un modelo para realizar algún tipo de predicción y evaluar el sistema en los mismos datos que se han utilizado para el entrenamiento es un **error metodológico**.

Un modelo que simplemente repite las etiquetas de los datos que acaba de ver durante el entrenamiento podría hacer un trabajo perfecto, pero ¿qué pasa con los datos que **no ha visto**? ¿El resultado corresponderá a algo útil? Esta situación se conoce como **sobreajuste** o **overfitting**.

Para evitar este tipo de problemas y tener una mejor idea del **comportamiento real** del sistema (ya sea de clasificación o regresión) al procesar datos **desconocidos**, se **divide el conjunto de datos en dos subconjuntos**: `X_train` y `X_test`. Esto permite que el sistema automatizado se **entrene** usando los datos de `X_train` y se **evalúe** con los datos de `X_test`.

---

Sin embargo, todavía existe el riesgo de que el modelo se **sobreentrene en el conjunto de prueba**, ya que los parámetros pueden ajustarse una y otra vez hasta que el estimador alcance un rendimiento óptimo en ese conjunto. De esta forma, el conocimiento sobre el conjunto de prueba puede “**filtrarse**” en el modelo, y las métricas de evaluación dejan de reflejar el **rendimiento de generalización**.

Para resolver este problema, se puede reservar **otra parte adicional** del conjunto de datos como un llamado **“conjunto de validación”**. El proceso es el siguiente:
1. El entrenamiento se realiza sobre el conjunto de entrenamiento (`train`).
2. La evaluación intermedia se lleva a cabo en el conjunto de validación (`validation`).
3. Cuando el experimento parece exitoso, se realiza la evaluación **final** en el conjunto de prueba (`test`).

---

No obstante, al **dividir los datos disponibles en tres subconjuntos**, se reduce de manera significativa la cantidad de muestras que pueden utilizarse para **aprender el modelo**, y los resultados pueden depender mucho de una **elección aleatoria particular** de los subconjuntos de entrenamiento y validación.

---

Una solución a este problema es un procedimiento llamado **validación cruzada** (o **cross-validation**, abreviado **CV**).



Hay diferentes estrategias; utilizaremos las herramientas disponibles en scikit-learn para este propósito.

https://scikit-learn.org/stable/modules/cross_validation.html

## Using the IRIS database


In [1]:
#load tools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
##Load iris
from sklearn.datasets import load_iris
import pandas as pd

# Cargar el dataset Iris
iris = load_iris()

# Convertirlo en un DataFrame para mejor visualización
df_iris = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# Agregar la columna de la clase (target)
df_iris['target'] = iris.target


In [3]:
#Verify data
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
df_iris['target'] 

0      0
1      0
2      0
3      0
4      0
      ..
145    2
146    2
147    2
148    2
149    2
Name: target, Length: 150, dtype: int32

## K-Fold  
`KFold` divide todas las muestras en **$k$ grupos de muestras**, llamados **folds** (si $k = n$, esto es equivalente a la estrategia de **Leave One Out**), de tamaños iguales (si es posible).  
La función de predicción se aprende utilizando **$k - 1$ folds**, y el fold que se deja fuera se utiliza para **prueba**.


<img src="https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_006.png" width="450">

There is a problem.....

In [6]:
from sklearn.model_selection import KFold

X = np.array(df_iris.drop(columns=['target']))
Y = np.array(df_iris['target'])
print(X.shape)
print(Y.shape)

kf = KFold(n_splits=10,shuffle=True)
for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        print("test labels: ", y_test)
        #print("train labels: ", y_train)
        

(150, 4)
(150,)
test labels:  [0 0 0 0 0 1 1 1 2 2 2 2 2 2 2]
test labels:  [0 0 0 0 0 0 0 1 1 1 1 2 2 2 2]
test labels:  [0 0 0 0 1 1 1 1 1 1 2 2 2 2 2]
test labels:  [0 0 0 1 1 1 1 1 1 2 2 2 2 2 2]
test labels:  [0 0 0 0 0 0 1 1 1 1 1 1 1 1 2]
test labels:  [0 0 0 1 1 1 1 2 2 2 2 2 2 2 2]
test labels:  [0 0 0 0 0 0 0 1 1 1 1 1 2 2 2]
test labels:  [0 0 0 0 0 1 1 1 2 2 2 2 2 2 2]
test labels:  [0 0 0 1 1 1 1 1 2 2 2 2 2 2 2]
test labels:  [0 0 0 0 0 0 0 1 1 1 1 1 1 2 2]



**Para resolver el problema, podemos mezclar (shuffle) el conjunto de datos antes de aplicar el algoritmo anterior. Usamos una permutación aleatoria de los índices de la siguiente manera:** 


In [7]:
#values from 10 to 20
x=np.arange(10,20)
print("valores de x:              ", x)
#shufle
y=np.random.permutation(10)
print("nuevos indices para x:     ", y)

#use the new indexes
x=x[y]
print("valores de x, desordenados:", x)

valores de x:               [10 11 12 13 14 15 16 17 18 19]
nuevos indices para x:      [4 3 6 5 1 9 0 7 8 2]
valores de x, desordenados: [14 13 16 15 11 19 10 17 18 12]


In [8]:
#Do the same for the IRIS dataset
ind=np.random.permutation(Y.size)

X=X[ind,:]
Y=Y[ind]

In [9]:
kf = KFold(n_splits=10)

for train_index, test_index in kf.split(X):
        #print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        print("test labels: ", y_test)

test labels:  [2 1 2 1 0 2 0 0 2 2 0 0 1 1 2]
test labels:  [0 1 1 2 0 0 1 1 0 1 2 1 1 2 1]
test labels:  [2 2 2 2 0 0 2 2 2 1 1 0 0 0 2]
test labels:  [0 2 1 1 0 1 1 1 2 2 0 1 1 0 2]
test labels:  [0 2 2 2 2 0 0 2 2 2 0 1 0 2 1]
test labels:  [2 2 2 0 0 0 1 1 1 1 2 0 0 0 2]
test labels:  [2 1 0 1 2 2 1 2 2 1 0 1 1 2 2]
test labels:  [0 1 1 1 0 0 1 1 2 1 1 0 1 1 2]
test labels:  [2 0 1 0 2 0 2 1 2 0 2 0 1 0 1]
test labels:  [0 0 2 0 0 1 0 0 1 0 1 0 0 2 1]


Hagamos un experimento

1. Logistic regression
2. KNN

In [10]:
#generamos dos vectores de ceros para guardar la tasa de acierto (% de muestras clasificadas correctamente) de
#los dos clasificadors, uno pada caso
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#numero de folds
k=10 

acc1 = []
acc2 = []

kf = KFold(n_splits=k)

for train_index, test_index in kf.split(X):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        #CLF 1
        clf1 = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf1.fit(X_train, y_train)
        #evaluate
        yp1 = clf1.predict(X_test)
        acc1.append(np.sum(yp1==y_test)/y_test.size*100)
        
        #Clf2
        clf2 = KNeighborsClassifier(n_neighbors=3)
        clf2.fit(X_train, y_train)
        
        #Evaluate
        yp2 = clf2.predict(X_test)
        acc2.append(np.sum(yp2==y_test)/y_test.size*100)
                
acc1=np.array(acc1)
acc2=np.array(acc2)


print("Logistic regression: average = %f, std = %f"% (acc1.mean(), acc1.std()))
print("KNN                : average = %f, std = %f"% (acc2.mean(), acc2.std()))

Logistic regression: average = 96.000000, std = 6.798693
KNN                : average = 96.000000, std = 5.333333


## Leave One Out (LOO)

**LeaveOneOut** (o **LOO**) es una técnica simple de **validación cruzada**.  
Cada conjunto de entrenamiento se crea tomando **todas las muestras excepto una**, siendo el conjunto de prueba la **muestra que se deja fuera**.  
Así, para **$n$ muestras**, tenemos **$n$ conjuntos de entrenamiento diferentes** y **$n$ conjuntos de prueba diferentes**.  
Este procedimiento de validación cruzada **no desperdicia muchos datos**, ya que solo se **elimina una muestra** del conjunto de entrenamiento:


In [11]:
from sklearn.model_selection import LeaveOneOut
acc1 = []
acc2 = []

X = np.array(df_iris.drop(columns=['target']))
Y = np.array(df_iris['target'])
print(X.shape)
print(Y.shape)

#index generator
loo = LeaveOneOut()

for train, test in loo.split(X):
        X_train, X_test = X[train], X[test]
        y_train, y_test = Y[train], Y[test]
        #Clf 1
        clf1 = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf1.fit(X_train, y_train)
        #eval
        yp1 = clf1.predict(X_test)
        acc1.append(yp1==y_test)
        
        #clf2
        clf2 = KNeighborsClassifier(n_neighbors=3)
        clf2.fit(X_train, y_train)
        
        #eval
        yp2 = clf2.predict(X_test)
        acc2.append(yp2==y_test)
                
acc1=np.array(acc1).sum()/len(acc1)*100
acc2=np.array(acc2).sum()/len(acc2)*100



print("Logistic regression: accuracy = ", acc1)
print("KNN                : accuracy = ", acc2)

(150, 4)
(150,)
Logistic regression: accuracy =  96.66666666666667
KNN                : accuracy =  96.0


Se puede notar que la tasa de acierto es igual a la que se obtuvo con KFOLDS, sin embargo en este caso no es posible calcular un promedio o una desviación estandard, pues en cada iteración solo habia una muestra, por lo que el acierto es 100% si esa muestra se clasifica bien, o 0% si se clasifica mal

## Validación cruzada aleatoria = Shuffle & Split

El iterador **ShuffleSplit** generará un número definido por el usuario de divisiones **independientes** del conjunto de datos en **entrenamiento** y **prueba**.  
Primero se mezclan (shuffle) las muestras y luego se dividen en un par de conjuntos: uno de entrenamiento y otro de prueba.

Es posible **controlar la aleatoriedad** para obtener resultados **reproducibles**, estableciendo explícitamente la semilla en el generador de números pseudoaleatorios mediante el parámetro `random_state`.

```python
ShuffleSplit(n_splits=20, test_size=0.3, random_state=0)
```

- **n_splits**: número de divisiones a realizar.  
- **test_size**: proporción de los datos que se usará para prueba.  

Por ejemplo: **70% - 30%** (entrenamiento - prueba).



In [12]:
from sklearn.model_selection import ShuffleSplit

acc1 = []
acc2 = []

X = np.array(df_iris.drop(columns=['target']))
Y = np.array(df_iris['target'])
print(X.shape)
print(Y.shape)


#
ss = ShuffleSplit(n_splits=10, test_size=0.3, random_state=0)

for train_index, test_index in ss.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        #clf1
        clf1 = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf1.fit(X_train, y_train)
        #evaluate
        yp1 = clf1.predict(X_test)
        acc1.append(np.sum(yp1==y_test)/y_test.size*100)
        
        #clf2
        clf2 = KNeighborsClassifier(n_neighbors=3)
        clf2.fit(X_train, y_train)
        
        #Evaluate
        yp2 = clf2.predict(X_test)
        acc2.append(np.sum(yp2==y_test)/y_test.size*100)

               
acc1=np.array(acc1)
acc2=np.array(acc2)

print("Logistic regression: Average = %f, std = %f"% (acc1.mean(), acc1.std()))
print("KNN                : Average = %f, std = %f"% (acc2.mean(), acc2.std()))    
del X, Y

(150, 4)
(150,)
Logistic regression: Average = 96.666667, std = 3.022549
KNN                : Average = 96.000000, std = 3.265986


## Iteradores de validación cruzada con estratificación basada en las etiquetas de clase.

Algunos problemas de clasificación pueden presentar un **gran desbalance** en la distribución de las clases objetivo; por ejemplo, podría haber **muchas más muestras negativas que positivas**.  
En estos casos, se recomienda utilizar **muestreo estratificado**, como el que se implementa en **StratifiedKFold** y **StratifiedShuffleSplit**, para asegurarse de que las **frecuencias relativas de cada clase se mantengan aproximadamente iguales** en cada fold de entrenamiento y validación.


### Stratified k-fold


**StratifiedKFold** es una variación de **k-fold** que devuelve folds **estratificados**: cada conjunto contiene aproximadamente el **mismo porcentaje de muestras de cada clase objetivo** que el conjunto completo.


In [13]:
from sklearn.datasets import fetch_openml
import pandas as pd


diabetes_data = fetch_openml(name='diabetes', version=1, as_frame=True)
df_diabetes = diabetes_data.frame

#  Separar X (features) e Y (target)
X = df_diabetes.drop(columns=['class'])  # Las características
y = df_diabetes['class']                 # La etiqueta objetivo (diabetes: tested_positive / tested_negative)

# Mostrar dimensiones de los datos
print(f"Dimensión de X: {X.shape}")
print(f"Dimensión de y: {y.shape}")

y_numeric = y.map({'tested_negative': 0, 'tested_positive': 1})

# Verificación
print(y_numeric.value_counts())
print(y_numeric.head())

Dimensión de X: (768, 8)
Dimensión de y: (768,)
class
0    500
1    268
Name: count, dtype: int64
0    1
1    0
2    1
3    0
4    1
Name: class, dtype: category
Categories (2, int64): [0, 1]


In [14]:
#Stratified k-fold

from sklearn.model_selection import StratifiedKFold, KFold

X = np.array(df_diabetes.drop(columns=['class']) )
Y = y_numeric
print(X.shape)
print(Y.shape)


acc1 = []
acc2 = []

#se genera el generador de indices de forma estratificada
skf = StratifiedKFold(n_splits=10)
   
for train_index, test_index in skf.split(X,Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        #clasficador 1
        clf1 = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf1.fit(X_train, y_train)
        #evaluar clf1 y guardar el resultado en acc1
        yp1 = clf1.predict(X_test)
        acc1.append(np.sum(yp1==y_test)/y_test.size*100)
        
        #clasificador 2
        clf2 = KNeighborsClassifier(n_neighbors=3)
        clf2.fit(X_train, y_train)
        
        #evaluar clf2 y guardar el resultado en acc2
        yp2 = clf2.predict(X_test)
        acc2.append(np.sum(yp2==y_test)/y_test.size*100)
                
acc1=np.array(acc1)
acc2=np.array(acc2)

print("Comparación de rendimiento de los dos clasificadores:\n")        
print("Logistic regression: promedio = %f, std = %f"% (acc1.mean(), acc1.std()))
print("KNN                : promedio = %f, std = %f"% (acc2.mean(), acc2.std()))
    

(768, 8)
(768,)
Comparación de rendimiento de los dos clasificadores:

Logistic regression: promedio = 77.347915, std = 3.574822
KNN                : promedio = 70.305878, std = 3.763358


### Stratified Shuffle Split

**StratifiedShuffleSplit** es una variación de **ShuffleSplit**, que devuelve divisiones **estratificadas**, es decir, crea particiones **preservando el mismo porcentaje de cada clase objetivo** que hay en el conjunto completo.



In [17]:
#Stratified ShuffleSplit 
from sklearn.model_selection import StratifiedShuffleSplit


X = np.array(df_diabetes.drop(columns=['class']) )
Y = y_numeric
print(X.shape)
print(Y.shape)


acc1 = []
acc2 = []

#se genera el generador de indices de forma estratificada
sss = StratifiedShuffleSplit(n_splits=100,test_size=0.3)
   
for train_index, test_index in sss.split(X,Y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        #clasficador 1
        clf1 = LogisticRegression(solver='lbfgs', max_iter=1000)
        clf1.fit(X_train, y_train)
        #evaluar clf1 y guardar el resultado en acc1
        yp1 = clf1.predict(X_test)
        acc1.append(np.sum(yp1==y_test)/y_test.size*100)
        
        #clasificador 2
        clf2 = KNeighborsClassifier(n_neighbors=3)
        clf2.fit(X_train, y_train)
        
        #evaluar clf2 y guardar el resultado en acc2
        yp2 = clf2.predict(X_test)
        acc2.append(np.sum(yp2==y_test)/y_test.size*100)
                
acc1=np.array(acc1)
acc2=np.array(acc2)

print("Comparación de rendimiento de los dos clasificadores:\n")        
print("Logistic regression: promedio = %f, std = %f"% (acc1.mean(), acc1.std()))
print("KNN                : promedio = %f, std = %f"% (acc2.mean(), acc2.std()))
    

(768, 8)
(768,)
Comparación de rendimiento de los dos clasificadores:

Logistic regression: promedio = 76.727273, std = 2.273548
KNN                : promedio = 69.774892, std = 2.221231
